In [10]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph
# https://blog.0x.org/0x-ecosystem-update-13/

In [11]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os
from datetime import datetime
import numpy as np
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
def client(url):
    sample_transport = RequestsHTTPTransport(url = url,
                                             verify = True,
                                             retries = 3)
    
    client = Client(transport=sample_transport)
    return client

In [13]:
# interesting tokens on MATIC

 #https://polygon.fulcrum.trade/borrow/asset
lending_tokens_fulcrum = ['LINK', 'BZRX', 'AAVE', 'WBTC', 'USDC', 'USDT', 'WETH', 'WMATIC']

# https://creamfinance.substack.com/p/cream-communications-14
lending_tokens_aave = ['MATIC', 'DAI', 'USDC', 'USDT', 'WETH', 'WBTC', 'AAVE']
lending_tokens_cream = ['USDC', 'USDT', 'DAI', 'WMATIC', 'WETH', 'WBTC', 'LINK', 'SUSHI', 'CRV', 'QUICK']
remaining_interesting_tokens = ['FRAX', 'ibBTC', 'ILV']

interesting_tokens = list(set(lending_tokens_fulcrum +\
                              lending_tokens_aave +\
                              lending_tokens_cream +\
                              remaining_interesting_tokens))

interesting_tokens

['USDC',
 'MATIC',
 'AAVE',
 'USDT',
 'LINK',
 'WETH',
 'QUICK',
 'ibBTC',
 'WBTC',
 'FRAX',
 'CRV',
 'SUSHI',
 'BZRX',
 'DAI',
 'WMATIC',
 'ILV']

## Quickswap

In [14]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02').execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

7192


In [15]:
try:
    df = pd.read_excel('pairdata_Quickswap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [16]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [17]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel('pairdata_Quickswap_MATIC.xlsx', index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,WMATIC-QUICK,WMATIC,QUICK
1,0x02e0e14d65175420fb12acd325a7be456bc22005,QUICK-DAI,QUICK,DAI
2,0x0554059d42e26f35cc958581c71fdfd92405d02f,WMATIC-AAVE,WMATIC,AAVE
3,0x0b3a76d33d93b2651dbf03b102eefa9500e89243,SUSHI-WMATIC,SUSHI,WMATIC
4,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,WETH-QUICK,WETH,QUICK


In [18]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [19]:
df_old = pd.read_csv("Quickswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-13 20:33:35.294230,8.820847e+08


In [20]:
df4.to_csv("Quickswap export.csv", index = False)

In [21]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(3)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-14 20:53:54.715938,8.838725e+08,2021-07-13 20:33:35.294230,8.820847e+08,0.002777,1.787754e+06,1.931629e+06
0x02e0e14d65175420fb12acd325a7be456bc22005,2021-07-14 20:53:54.926949,2.389270e+05,2021-07-13 20:33:35.498607,2.373375e+05,0.002777,1.589479e+03,1.717397e+03
0x0554059d42e26f35cc958581c71fdfd92405d02f,2021-07-14 20:53:55.132988,4.439976e+05,2021-07-13 20:33:35.718732,4.433726e+05,0.002777,6.249468e+02,6.752413e+02


In [22]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [23]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Quickswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,1.500136e+06,2021-07-14,1.934407e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,4500.406809,8.492,8.492,8.492,8.491,8.491,8.492,8.492,8.492,8.493
1,1.589250e+06,2021-07-13,1.927339e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,4767.751059,9.029,9.029,9.029,9.029,9.028,9.029,9.029,9.030,9.030


In [24]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,dailyVolumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xab4f7a3199b2cd7a8bfb425941ead08facb6d617,6.417838,2021-07-13,27.857985,[Quickswap MATIC] CRV-WETH,0.019254,25.227,3.084,1.331,0.684,0.347,-4.083,-1.488,-0.723,-0.356
0xfba51e1ea2d46c4d5e98f27e1f5333969d2222d7,0.142935,2021-07-13,10.664842,[Quickswap MATIC] LINK-DAI,0.000429,1.468,0.074,0.031,0.015,0.008,-0.083,-0.032,-0.016,-0.008


In [25]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x019ba0325f1988213d448b3472fa1cf8d07618d7,9.985642,9.985539,9.985384,9.985126,9.984610,9.985746,9.985900,9.986159,9.986675,1.934407e+07
0x02e0e14d65175420fb12acd325a7be456bc22005,26.177386,25.402977,24.323624,22.715049,20.061611,27.000495,28.337018,30.885031,37.657155,6.560612e+03


In [26]:

df_final_quickswap = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'dailyVolumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_quickswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                            
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                            
                              'APR timedelta Liq - 2000',                            
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                                            
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                                            
                              'APR yday Liq - 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_quickswap.head(1)

# df_final_quickswap_stylized = df_final_quickswap.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_quickswap_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-13,44.55026,44.537543,44.51848,44.486746,44.423412,44.562985,44.582086,44.613957,44.677835,49.711,49.697,49.675,49.639,49.567,49.726,49.747,49.784,49.856,700417.177317,687939.098085,312313.820173


In [27]:
# dfi.export(df_final_quickswap_stylized, 'Quickswap MATIC timedelta.png')
# os.system('xdg-open "Quickswap MATIC timedelta.png"')

## Sushiswap MATIC

In [28]:
# sushiswap_matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

# sample_transport=RequestsHTTPTransport(url = sushiswap_matic,
#                                        verify = True,
#                                        retries = 3)

# client = Client(transport=sample_transport)

In [29]:
query = gql('''
query {
  factories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

1561


In [30]:
try:
    df = pd.read_excel('pairdata_SushiSwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [31]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [32]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_SushiSwap_MATIC.xlsx", index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,WMATIC-CRV,WMATIC,CRV
1,0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,SUSHI-DAI,SUSHI,DAI
2,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX-USDT,FRAX,USDT
3,0x2813d43463c374a680f235c428fb1d7f08de0b69,WETH-AAVE,WETH,AAVE
4,0x29bb3ddf61c831adda06439740b340ae48606da6,WBTC-QUICK,WBTC,QUICK


In [33]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]
df4.head(1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

,id,timestamp,volumeUSD
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-14 20:56:16.633519,1361.915486


In [34]:
df_old = pd.read_csv("SushiSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-13 20:36:08.977055,1337.370145


In [35]:
df4.to_csv("SushiSwap export.csv", index = False)

In [36]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-14 20:56:16.633519,1361.915486,2021-07-13 20:36:08.977055,1337.370145,0.002776,24.545341,26.524262
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,2021-07-14 20:56:16.848719,10332.705095,2021-07-13 20:36:09.182262,10280.381309,0.002776,52.323786,56.542284


In [37]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

In [38]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['volumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Sushiswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-14,358.153811,11.466412,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.034399,3.506,2.249,1.463,0.924,0.532,7.939,-8.852,-1.956,-0.765
1,2021-07-13,645.320265,38.539765,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.115619,6.540,4.992,3.685,2.565,1.595,9.477,29.040,-11.898,-3.115


In [39]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-07-13,6.068985e+07,8.056780e+06,[Sushiswap MATIC] USDC-WETH,24170.339938,14.536,14.536,14.536,14.536,14.536,14.537,14.537,14.537,14.537
0x383be588327216586e131e63592a2dc976a16655,2021-07-13,1.117848e+03,1.011297e+03,[Sushiswap MATIC] WMATIC-QUICK,3.033892,99.063,84.029,68.447,52.288,35.517,120.649,179.230,939.663,-125.531


In [40]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,7.405830,4.752142,3.090852,1.952965,1.124789,16.771181,-18.699312,-4.132495,-1.615514,358.153811
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,12.939966,8.876918,6.034662,3.934858,2.320199,23.861680,-89.690613,-10.042295,-3.617453,436.958526


In [41]:
df_final_sushiswap_matic = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                                'APR timedelta Liq + 500',
                                'APR timedelta Liq + 1000',                                            
                                'APR timedelta Liq + 2000',
                                'APR timedelta Liq - 200',
                                'APR timedelta Liq - 500',
                                'APR timedelta Liq - 1000',                                            
                                'APR timedelta Liq - 2000',                           
                              'APR yday',
                             'APR yday Liq + 200',
                                'APR yday Liq + 500',
                                'APR yday Liq + 1000',                                            
                                'APR yday Liq + 2000',
                                'APR yday Liq - 200',
                                'APR yday Liq - 500',
                                'APR yday Liq - 1000',                                            
                                'APR yday Liq - 2000',                          
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_matic.head(1)

# df_final_sushiswap_matic_stylized = df_final_sushiswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_sushiswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] WMATIC-USDC,2021-07-13,58.538802,58.535868,58.531466,58.524132,58.509469,58.541737,58.54614,58.55348,58.568166,41.198,41.196,41.193,41.187,41.177,41.2,41.203,41.208,41.218,3.989226e+06,4.069802e+06,1.531197e+06


In [42]:
# dfi.export(df_final_stylized_sushi, 'SushiSwap MATIC timedelta.png')
# os.system('xdg-open "SushiSwap MATIC timedelta.png"')

## Honeyswap Polygon

In [43]:
query = gql('''
query {
  honeyswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(query)
number_of_pairs = int(response['honeyswapFactories'][0]['pairCount'])
print(number_of_pairs)

147


In [44]:
try:
    df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [45]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [46]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_HoneySwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x0c787944946d22922c9f41c477cc539700d35bb2,WETH-AAVE,WETH,AAVE
1,0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,WETH-USDT,WETH,USDT


In [47]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

,id,timestamp,volumeUSD
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-14 20:56:35.858171,98483.510026


In [48]:
df_old = pd.read_csv("HoneySwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-13 20:36:28.533129,91324.559791


In [49]:
df_timedelta.to_csv("HoneySwap export.csv", index = False)

In [50]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-14 20:56:35.858171,98483.510026,2021-07-13 20:36:28.533129,91324.559791,0.002776,7158.950236,6446.796796
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,2021-07-14 20:56:36.060688,177576.388853,2021-07-13 20:36:28.747114,168947.227512,0.002776,8629.161341,7770.756205


In [51]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

In [52]:
df_last_two_days2 = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days2[['volumeUSD','reserveUSD']] = df_last_two_days2[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days2['date'] = pd.to_datetime(df_last_two_days2['date'], unit='s').dt.date
df_last_two_days2['fee'] = round(df_last_two_days2['volumeUSD'] * 0.0025, 6)
df_last_two_days2['APR yday'] = round(100 * df_last_two_days2['fee'] * 365 / df_last_two_days2['reserveUSD'], 3)
df_last_two_days2['Pair'] = '[Honeyswap MATIC] ' + df_last_two_days2['Pair']
df_last_two_days2['APR yday Liq + 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 200), 3)
df_last_two_days2['APR yday Liq + 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 500), 3) 
df_last_two_days2['APR yday Liq + 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 1000), 3)                                                  
df_last_two_days2['APR yday Liq + 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 2000), 3)
df_last_two_days2['APR yday Liq - 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 200), 3)
df_last_two_days2['APR yday Liq - 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 500), 3)
df_last_two_days2['APR yday Liq - 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 1000), 3)                                                 
df_last_two_days2['APR yday Liq - 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 2000), 3)                                                 

df_last_two_days2.head(2)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,5465.057898,2021-07-14,154015.617316,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,13.662645,3.238,3.234,3.227,3.217,3.196,3.242,3.248,3.259,3.280
1,3388.363176,2021-07-13,153176.780726,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,8.470908,2.019,2.016,2.012,2.005,1.992,2.021,2.025,2.032,2.045


In [53]:
df_yesterday = df_last_two_days2.sort_values(by = 'date',
                                             ascending = False)\
                                .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                .reset_index(drop = True)\
                                .drop(columns = ['Token 0', 'Token 1'])\
                                .set_index('id')

df_yesterday.head(2)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc,0.218698,2021-07-13,15.973456,[Honeyswap MATIC] WMATIC-CRV,0.000547,1.25,0.092,0.039,0.02,0.01,-0.108,-0.041,-0.02,-0.01
0xe31442544066251d3e1d21295ea4b1737a7d8082,0.000000,2021-07-13,0.059915,[Honeyswap MATIC] WETH-QUICK,0.000000,0.00,0.000,0.000,0.00,0.00,-0.000,-0.000,-0.00,-0.00


In [54]:
df_combined2 = pd.concat([df_combined,
                          df_last_two_days2.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',
                             'APR timedelta Liq - 2000',                              
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x0c787944946d22922c9f41c477cc539700d35bb2,4.185807,4.180379,4.172262,4.158805,4.132148,4.191250,4.199440,4.213163,4.240878,154015.617316
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,6.135937,6.126262,6.111807,6.087866,6.040543,6.145642,6.160258,6.184773,6.234393,126643.349406


In [55]:
df_final_honeyswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_honeyswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_honeyswap_matic.head(1)

# df_final_honeyswap_matic_stylized = df_final_honeyswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_honeyswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Honeyswap MATIC] WMATIC-USDC,2021-07-13,23.943752,21.051493,17.822263,14.193528,10.086263,27.757323,36.470397,76.48506,-64.038317,17.195,15.113,12.789,10.18,7.231,19.944,26.233,55.298,-45.479,1455.712909,1451.280311,273.479214


## DoDoSwap Polygon

In [56]:
client_exe = client('https://api.thegraph.com/subgraphs/name/dodoex/dodoex-v2-polygon')
# https://thegraph.com/legacy-explorer/subgraph/dodoex/dodoex-v2-polygon
fee = 0.0025

In [57]:
query = gql('''
query {
  dodoZoos {
    pairCount
  }
}
''')

response = client_exe.execute(query)
number_of_pairs = int(response['dodoZoos'][0]['pairCount'])
print(number_of_pairs)

199


In [58]:
# # try:
# #     df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
# # except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      baseSymbol
                      quoteSymbol
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200


In [59]:
df = df.dropna()


df['Pair'] = df['baseSymbol'] + '-' + df['quoteSymbol']
df.columns = ['Token 0', 'id', 'Token 1', 'Pair']
df = df[['id','Pair', 'Token 0', 'Token 1']]
df.head(2)

,id,Pair,Token 0,Token 1
0,0x01577045e19463645a3043094bce5bc90291a51d,ONE-WMATIC,ONE,WMATIC
20,0x0d4856bca2035462947c479a857cfa50903c13ca,WMATIC-ONE,WMATIC,ONE


In [60]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_DoDoSwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
50,0x0f20c4148369fb70083593b1475922bec87aebe3,WMATIC-USDC,WMATIC,USDC
59,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,WMATIC-USDC,WMATIC,USDC


In [61]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
        lpFeeRate
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD', 'lpFeeRate']]
df_timedelta.head(1)

 1 / 4 pair: 0x0f20c4148369fb70083593b1475922bec87aebe3
 2 / 4 pair: 0x10dd6d8a29d489bede472cc1b22dc695c144c5c7
 3 / 4 pair: 0x1ea658dd919d77edccea1a7bfd417b048934ad6b
 4 / 4 pair: 0x2f7e940ea475cb424c8ba6025d48350faea48dc1


,id,timestamp,volumeUSD,lpFeeRate
0,0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-14 20:56:49.430774,66.169014,0.004


In [62]:
df_old = pd.read_csv("DoDoSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD', 'lpFeeRate'])
df_old = df_old[['id', 'timestamp', 'volumeUSD', 'lpFeeRate']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old', 'lpFeeRate']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old,lpFeeRate
0,0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-13 20:36:42.747076,66.169014,0.004


In [63]:
df_timedelta.to_csv("DoDoSwap export.csv", index = False)

In [64]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old').drop(columns = ['lpFeeRate'])], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['lpFeeRate'] = df_combined['lpFeeRate'].astype(float)
df_combined['fees per year'] = df_combined['volumeUSD delta'] * df_combined['lpFeeRate'] / df_combined['timedelta']
df_combined = df_combined.drop(columns = ['lpFeeRate'])
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-14 20:56:49.430774,6.616901e+01,2021-07-13 20:36:42.747076,66.169014,0.002776,0.000000,0.000000
0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,2021-07-14 20:56:49.623821,1.161237e+06,2021-07-13 20:36:42.954015,180600.648039,0.002776,980635.966545,423883.919262


In [65]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    pairAddress
    volumeUSD
    baseTokenReserve
    quoteTokenReserve
    baseUsdPrice
    quoteUsdPrice
    lpFeeRate
  }
}'''
    response = client_exe.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['baseTokenReserve',
                  'baseUsdPrice',
                  'lpFeeRate',
                  'quoteTokenReserve',
                  'quoteUsdPrice',
                  'volumeUSD']] = \
df_last_two_days[['baseTokenReserve',
                  'baseUsdPrice',
                  'lpFeeRate',
                  'quoteTokenReserve',
                  'quoteUsdPrice',
                  'volumeUSD']].astype(float)

df_last_two_days['reserveUSD'] = df_last_two_days['baseTokenReserve'] * df_last_two_days['baseUsdPrice'] + df_last_two_days['quoteTokenReserve'] * df_last_two_days['quoteUsdPrice']
df_last_two_days = df_last_two_days.drop(columns = ['baseTokenReserve',
                                                    'baseUsdPrice',
                                                    'quoteTokenReserve',
                                                    'quoteUsdPrice'])
df_last_two_days

 1 / 4 pair: 0x0f20c4148369fb70083593b1475922bec87aebe3
 2 / 4 pair: 0x10dd6d8a29d489bede472cc1b22dc695c144c5c7
 3 / 4 pair: 0x1ea658dd919d77edccea1a7bfd417b048934ad6b
 4 / 4 pair: 0x2f7e940ea475cb424c8ba6025d48350faea48dc1


,date,lpFeeRate,pairAddress,volumeUSD,id,reserveUSD
0,1626134400,0.0040,0x0f20c4148369fb70083593b1475922bec87aebe3,66.169014,0x0f20c4148369fb70083593b1475922bec87aebe3,742.454507
0,1626220800,0.0012,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,852611.555976,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,31629.756419
1,1626134400,0.0012,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,287622.077022,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,9991.388857
0,1626134400,0.0080,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,0.069275,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,0.133288
0,1626220800,0.0030,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,0.203287,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,3.025442
1,1626134400,0.0030,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,0.014032,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,6.964398


In [66]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days_incl_pairs[['volumeUSD','reserveUSD']] = df_last_two_days_incl_pairs[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days_incl_pairs['date'] = pd.to_datetime(df_last_two_days_incl_pairs['date'], unit='s').dt.date
df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * df_last_two_days_incl_pairs['lpFeeRate'], 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[DodoSwap MATIC] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3) 
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)                                                  
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)                                                 
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)                                                 

df_last_two_days_incl_pairs.head(2)

,date,lpFeeRate,pairAddress,volumeUSD,id,reserveUSD,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-13,0.0040,0x0f20c4148369fb70083593b1475922bec87aebe3,66.169014,0x0f20c4148369fb70083593b1475922bec87aebe3,742.454507,[DodoSwap MATIC] WMATIC-USDC,WMATIC,USDC,0.264676,13.012,10.251,7.775,5.544,3.523,17.809,39.845,-37.511,-7.682
1,2021-07-14,0.0012,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,852611.555976,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,31629.756419,[DodoSwap MATIC] WMATIC-USDC,WMATIC,USDC,1023.133867,1180.673,1173.254,1162.299,1144.489,1110.457,1188.186,1199.636,1219.219,1260.368


In [67]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,lpFeeRate,pairAddress,volumeUSD,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,,,
0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,2021-07-13,0.0012,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,287622.077022,9991.388857,[DodoSwap MATIC] WMATIC-USDC,345.146492,1260.87,1236.127,1200.78,1146.156,1050.574,1286.625,1327.292,1401.101,1576.428


In [68]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_DoDoSwap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_DoDoSwap_matic.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_DoDoSwap_matic.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[DodoSwap MATIC] WMATIC-USDC,2021-07-13,1340.14285,1331.722159,1319.287684,1299.071663,1260.443025,1348.670711,1361.66796,1383.895822,1430.602106,1260.87,1236.127,1200.78,1146.156,1050.574,1286.625,1327.292,1401.101,1576.428,31629.756419,9991.388857,287622.077022


## Ape Swap Polygon

In [135]:
client_exe = client('https://api.thegraph.com/subgraphs/name/apeswapfinance/dex-polygon') # https://thegraph.com/legacy-explorer/subgraph/apeswapfinance/dex-polygon
fee = 0.002

query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:


167


In [136]:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

83
83


,id,token0,token1
0,0x019011032a7ac3a87ee885b6c08467ac46ad11cd,{'symbol': 'WMATIC'},{'symbol': 'USDC'}


In [137]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)



,id,Pair,Token 0,Token 1
7,0x110bc65553bb8013b9db639fe269cc4ba5d3a9a9,USDC-SNX,USDC,SNX


In [138]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 3 pair: 0x110bc65553bb8013b9db639fe269cc4ba5d3a9a9
 2 / 3 pair: 0x237ac473ac0ae4551019d9298a4118b3144f26a8
 3 / 3 pair: 0x5b13b583d4317ab15186ed660a1e4c65c10da659


,id,timestamp,volumeUSD
0,0x110bc65553bb8013b9db639fe269cc4ba5d3a9a9,2021-07-14 21:24:09.949941,962348.145035


In [139]:
df_old = pd.read_csv("ApeSwap export MATIC.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("ApeSwap export MATIC.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 3 pair: 0x110bc65553bb8013b9db639fe269cc4ba5d3a9a9
 2 / 3 pair: 0x237ac473ac0ae4551019d9298a4118b3144f26a8
 3 / 3 pair: 0x5b13b583d4317ab15186ed660a1e4c65c10da659


,volumeUSD,date,reserveUSD,id
0,7935.007846,2021-07-14,65607.373274,0x110bc65553bb8013b9db639fe269cc4ba5d3a9a9


In [140]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[ApeSwap MATIC] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_apeswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_apeswap_matic.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_apeswap_matic.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[ApeSwap MATIC] USDC-SNX,2021-07-13,46.271967,46.131339,45.921991,45.57727,44.903123,46.413456,46.627318,46.98817,47.726892,7.405,7.383,7.351,7.297,7.192,7.427,7.46,7.516,7.631,65607.373274,67566.335973,6853.755349


## Balancer Polygon

In [69]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2')
# #https://thegraph.com/explorer/subgraph/balancer-labs/balancer-polygon-v2
# fee = 0.0025

In [70]:
# query = gql('''
#             query {
#                   balancers {
#                     poolCount
#                   }
#                 }
#             ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['balancers'][0]['poolCount'])
# print(number_of_pairs)

In [71]:
# # make a list of ids and pairs on balancer

# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pools(first: '''+str(100)+' skip: '+str(i)+'''){
#                       tokens{symbol}
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pools']))
#             print(len(df))
#         except:
#             pass
    
# df['tokens'] = df['tokens'].astype(str).str.replace("{'symbol': ",'', regex= True)\
#                                            .replace("}",'', regex= True)\
#                                            .replace("]",'', regex= True)\
#                                            .replace('\[', '', regex = True)\
#                                            .replace("'",'', regex= True)\
#                                            .replace(" ",'', regex= True)

# number_different_tokens = len(df['tokens'].str.split(',', expand = True).columns)
# column_list = []

# for token_column in range(number_different_tokens):
#     column_list.append('Token' + str(token_column + 1))
    
# df[column_list] = df['tokens'].str.split(',', expand = True)
# df.replace()

In [72]:
# # make a list of all tokens on Balancer that are not in my interesting tokens

# df_tokens = pd.DataFrame([])

# for column in column_list:
#     df_tokenpart = df[[column]]
#     df_tokenpart = df_tokenpart.rename(columns = {column:'Token'})
#     df_tokens = df_tokens.append(df_tokenpart)
    
# all_balancer_tokens = df_tokens.drop_duplicates().dropna().sort_values(by = 'Token')['Token'].unique().tolist()
# tokens_to_avoid = set(all_balancer_tokens) - set(interesting_tokens)

In [73]:
# # filter out the pairs on balancer that contain tokens to avoid
# df_filtered = df.copy()

# for column in column_list:
#     df_filtered = df_filtered.loc[~df[column].isin(tokens_to_avoid)]
    
# df_filtered = df_filtered[['id', 'tokens']]
# df_filtered

In [74]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pools(where: {id:"''' + id + '''"}){
#         totalSwapVolume
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pools'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['totalSwapVolume'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [75]:
# df_old = pd.read_csv("Balancer export MATIC.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [76]:
# df_timedelta.to_csv("Balancer export MATIC.csv", index = False)

In [77]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

# df_combined.head(2)

In [78]:
# querystring = """
#               query {
#                     poolSnapshots(orderBy: timestamp, orderDirection: desc){
#                       timestamp
#                     }
#                     }
#                 """

# response = client_exe.execute(gql(querystring))

# df_time = pd.DataFrame.from_dict(response['poolSnapshots'])\
#                       .drop_duplicates()\
#                       .sort_values(by = 'timestamp', ascending = False).head(2)

# df_time['timestamp'].values.tolist()

In [79]:
# {
#   pools{
#     id
#     totalSwapVolume
#     totalLiquidity
#   }
# }

In [80]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   amounts
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
# }
# }
# """

In [81]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
#   pool{historicalValues(orderBy: block, orderDirection: desc first: 2){
#     poolLiquidity
#     poolId {
#       id
#     }
#     block
#     id
#   }}
# }
# }
# """

In [82]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     dailyVolumeUSD
#     reserveUSD
#   }
# }'''
#     response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## FTM settings

In [83]:
lending_tokens_cream = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI', 'YFI', 'SNX', 'BAND', 'AAVE', 'COVER', 'CREAM', 'HEGIC']
remaining_interesting_tokens = ['WFTM', 'FTM', 'USDC', 'DAI', 'ETH', 'fUSDT', 'FETH', 'WBTC', 'FRAX', 'FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_cream +\
                              remaining_interesting_tokens))

## SpiritSwap

In [84]:
query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

738


In [85]:
try:
    df = pd.read_excel('pairdata_Spiritswap_FTM.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0{symbol}
                          token1{symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [86]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [87]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_Spiritswap_FTM.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,WFTM-CREAM,WFTM,CREAM
1,0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,WFTM-FRAX,WFTM,FRAX


In [88]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,id,timestamp,volumeUSD
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-14 20:56:52.171390,4.530421e+06


In [89]:
df_old = pd.read_csv("Spiritswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-13 20:36:45.438935,4.519586e+06


In [90]:
df_timedelta.to_csv("Spiritswap export FTM.csv", index = False)

In [91]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']

df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-14 20:56:52.171390,4.530421e+06,2021-07-13 20:36:45.438935,4.519586e+06,0.002776,10835.398911,11709.105696
0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,2021-07-14 20:56:52.397256,1.083959e+07,2021-07-13 20:36:45.645475,1.080522e+07,0.002776,34376.539029,37148.465273


In [92]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,volumeUSD,date,reserveUSD,id
0,4866.440232,2021-07-14,223264.407229,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a


In [93]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * 0.003, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpiritSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,4866.440232,2021-07-14,223264.407229,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[SpiritSwap FTM] WFTM-CREAM,WFTM,CREAM,14.599321,2.387,2.385,2.381,2.376,2.366,2.389,2.392,2.397,2.408


In [94]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,16180.026856,2021-07-13,225167.116885,[SpiritSwap FTM] WFTM-CREAM,48.540081,7.868,7.861,7.851,7.834,7.799,7.875,7.886,7.904,7.939


In [95]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,5.244502,5.239808,5.232783,5.221116,5.197939,5.249204,5.256273,5.268098,5.291907,223264.407229


In [96]:
# df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
#                                  axis = 1)[['Pair',
#                                             'date',
#                                             'APR timedelta',
#                                             'APR timedelta Liq + 200',
#                                             'APR timedelta Liq + 500',
#                                             'APR timedelta Liq + 1000',                                            
#                                             'APR timedelta Liq + 2000',
#                                             'APR timedelta Liq - 200',
#                                             'APR timedelta Liq - 500',
#                                             'APR timedelta Liq - 1000',                                            
#                                             'APR timedelta Liq - 2000',                                             
#                                             'APR yday',
#                                             'APR yday Liq + 200',
#                                             'APR yday Liq + 500',
#                                             'APR yday Liq + 1000',                                            
#                                             'APR yday Liq + 2000',
#                                             'APR yday Liq - 200',
#                                             'APR yday Liq - 500',
#                                             'APR yday Liq - 1000',                                            
#                                             'APR yday Liq - 2000',                                            
#                                             'reserveUSD',
#                                             'volumeUSD']].set_index('Pair')\
#                        .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

# df_final_spiritswap.columns = ['date',
#                               'APR timedelta',
#                                 'APR timedelta',
#                                 'APR timedelta Liq + 200',
#                                 'APR timedelta Liq + 500',
#                                 'APR timedelta Liq + 1000',                                            
#                                 'APR timedelta Liq + 2000',
#                                 'APR timedelta Liq - 200',
#                                 'APR timedelta Liq - 500',
#                                 'APR timedelta Liq - 1000',                                            
#                                 'APR timedelta Liq - 2000',     
#                               'APR yday',
#                                 'APR yday Liq + 200',
#                                 'APR yday Liq + 500',
#                                 'APR yday Liq + 1000',                                            
#                                 'APR yday Liq + 2000',
#                                 'APR yday Liq - 200',
#                                 'APR yday Liq - 500',
#                                 'APR yday Liq - 1000',                                            
#                                 'APR yday Liq - 2000', 
#                               'Liquidity timedelta',
#                               'Liquidity yday',
#                               'Volume yday']

# df_final_spiritswap.head(1)

df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spiritswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

## SpookySwap

In [97]:
client_exe = client('https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange')
fee = 0.002

In [98]:
query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

564


In [99]:
# try:
#     df = pd.read_excel('pairdata_Spookyswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
          query {
          pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                  token0 {symbol}
                  token1 {symbol}
                  id
              }
            }
            '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200
300
305
305
305


In [100]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
5,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM-SNX,WFTM,SNX


In [101]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,id,timestamp,volumeUSD
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-14 20:57:09.390413,6.094239e+06


In [102]:
df_old = pd.read_csv("Spookyswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-13 20:37:01.476505,5.990256e+06


In [103]:
df_timedelta.to_csv("Spookyswap export.csv", index = False)

In [104]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-14 20:57:09.390413,6.094239e+06,2021-07-13 20:37:01.476505,5.990256e+06,0.002776,103983.180396,74910.863327


In [105]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,volumeUSD,date,reserveUSD,id
0,84871.177887,2021-07-14,1.001963e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7


In [106]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpookySwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,84871.177887,2021-07-14,1.001963e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[SpookySwap FTM] WFTM-SNX,WFTM,SNX,169.742356,6.183,6.182,6.18,6.177,6.171,6.185,6.187,6.19,6.196


In [107]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xe120ffbda0d14f3bb6d6053e90e63c572a66a428,309555.244113,2021-07-13,3.020885e+06,[SpookySwap FTM] WFTM-DAI,619.110488,7.48,7.48,7.479,7.478,7.475,7.481,7.482,7.483,7.485


In [108]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,7.47641,7.474918,7.472681,7.468956,7.461517,7.477903,7.480143,7.48388,7.491364,1.001963e+06


In [109]:
df_final_spookyswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spookyswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                               
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                               
                              'APR timedelta Liq - 2000',                               
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                               
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                               
                              'APR yday Liq - 2000',                               
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spookyswap.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SpookySwap FTM] USDC-WFTM,2021-07-13,29.829478,29.828174,29.826218,29.822958,29.816442,29.830782,29.832739,29.836,29.842525,23.575,23.574,23.572,23.57,23.564,23.576,23.578,23.58,23.585,4.574501e+06,4.465477e+06,1.442096e+06


## SushiSwap FTM

In [110]:
client_exe = client('https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange')
fee = 0.003

In [111]:
query = gql('''
            query {factories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

720


In [112]:
# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

100
183
183
183
183
183
183
183


,id,token0,token1
0,0x02f07a24ec8e6b34514f2cc8282af7d67f55d010,{'symbol': 'WFTM'},{'symbol': 'eBURN'}


In [113]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
16,0x147e27be8e66f74c7659d88e2e6a13188873d58b,WFTM-CREAM,WFTM,CREAM


In [114]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 18 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 18 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 18 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 18 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 18 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 18 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 7 / 18 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
 8 / 18 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
 9 / 18 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
10 / 18 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
11 / 18 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
12 / 18 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
13 / 18 pair: 0xc92d97416328a461bce2f539498b505c6c97748f
14 / 18 pair: 0xd019dd7c760c6431797d6ed170bffb8faee11f99
15 / 18 pair: 0xd32f2eb49e91aa160946f3538564118388d6246a
16 / 18 pair: 0xe74a6b1f2034c8c137610cafad49a6763cd25865
17 / 18 pair: 0xed88d84c8728baaa53fd7226c9e55bd36e777ade
18 / 18 pair: 0xfe69403cf2e2239

,id,timestamp,volumeUSD
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-14 20:57:44.099803,2.397137e+06


In [115]:
df_old = pd.read_csv("Sushiswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-13 20:37:17.784547,4.067096e+06


In [116]:
df_timedelta.to_csv("Sushiswap export FTM.csv", index = False)

In [117]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-14 20:57:44.099803,2.397137e+06,2021-07-13 20:37:17.784547,4.067096e+06,0.002777,-1.669959e+06,-1.804212e+06


In [118]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pair:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            volumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 18 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 18 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 18 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 18 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 18 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 18 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 7 / 18 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
 8 / 18 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
 9 / 18 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
10 / 18 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
11 / 18 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
12 / 18 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
13 / 18 pair: 0xc92d97416328a461bce2f539498b505c6c97748f
14 / 18 pair: 0xd019dd7c760c6431797d6ed170bffb8faee11f99
15 / 18 pair: 0xd32f2eb49e91aa160946f3538564118388d6246a
16 / 18 pair: 0xe74a6b1f2034c8c137610cafad49a6763cd25865
17 / 18 pair: 0xed88d84c8728baaa53fd7226c9e55bd36e777ade
18 / 18 pair: 0xfe69403cf2e2239

,date,reserveUSD,volumeUSD,id
0,2021-07-14,125643.09209,1065.205125,0x147e27be8e66f74c7659d88e2e6a13188873d58b


In [119]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SushiSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-14,125643.09209,1065.205125,0x147e27be8e66f74c7659d88e2e6a13188873d58b,[SushiSwap FTM] WFTM-CREAM,WFTM,CREAM,3.195615,0.928,0.927,0.925,0.921,0.914,0.93,0.932,0.936,0.943


In [120]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x2d89bb3b1448152002453aa37fbe4051afdbf56d,2021-07-13,79763.380944,5439.74775,[SushiSwap FTM] USDC-AAVE,16.319243,7.468,7.449,7.421,7.375,7.285,7.487,7.515,7.563,7.66


In [121]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x147e27be8e66f74c7659d88e2e6a13188873d58b,-1435.981893,-1433.699715,-1430.290016,-1424.643084,-1413.481938,-1438.271349,-1441.719253,-1447.502643,-1459.209748,125643.09209


In [122]:
df_final_sushiswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SushiSwap FTM] USDC-YFI,2021-05-09,-20221.283361,-1.017088,-0.406847,-0.203426,-0.101713,1.01719,0.406864,0.20343,0.101714,222695.279,11.201,4.481,2.24,1.12,-11.202,-4.481,-2.24,-1.12,0.01006,0.01006,20.459656


## WakaSwap FTM

In [123]:
client_exe = client('https://api.thegraph.com/subgraphs/name/waka-finance/waka-graph') # https://thegraph.com/explorer/subgraph/waka-finance/waka-graph
fee = 0.003

query = gql('''
            query {wakaSwapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['wakaSwapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

36
24
 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,id,timestamp,volumeUSD
0,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb,2021-07-14 20:58:02.969775,7.241918e+06


In [124]:
df_old = pd.read_csv("WakaSwap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,volumeUSD,date,reserveUSD,id
0,3553.297331,2021-07-14,117188.155655,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb


In [125]:
df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

In [126]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[WakaSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_wakaswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_wakaswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_wakaswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[WakaSwap FTM] USDC-WFTM,2021-07-13,20.33767,20.31129,20.271848,20.206451,20.076914,20.364119,20.403921,20.470604,20.605288,9.951,9.938,9.919,9.887,9.823,9.964,9.983,10.016,10.082,153990.263353,154276.473562,14019.894592


In [141]:
pd.set_option('display.max_columns', None)

df_final_combined = df_final_sushiswap_matic.append(df_final_quickswap)\
                                            .append(df_final_honeyswap_matic)\
                                            .append(df_final_wakaswap_ftm)\
                                            .append(df_final_spiritswap)\
                                            .append(df_final_spookyswap)\
                                            .append(df_final_sushiswap_ftm)\
                                            .append(df_final_DoDoSwap_matic)\
                                            .append(df_final_apeswap_matic)\
                                            .sort_values(by = 'APR timedelta Liq + 200',
                                                         ascending = False)

df_final_combined[['APR timedelta',
                   'APR timedelta Liq + 200',
                   'APR timedelta Liq + 500',
                   'APR timedelta Liq + 1000',                   
                   'APR timedelta Liq + 2000',
                   'APR timedelta Liq - 200',
                   'APR timedelta Liq - 500',
                   'APR timedelta Liq - 1000',                   
                   'APR timedelta Liq - 2000',                   
                   'APR yday',
                   'APR yday Liq + 200',
                   'APR yday Liq + 500',
                   'APR yday Liq + 1000',                   
                   'APR yday Liq + 2000',
                   'APR yday Liq - 200',
                   'APR yday Liq - 500',
                   'APR yday Liq - 1000',                   
                   'APR yday Liq - 2000']] = df_final_combined[['APR timedelta',
                                                                'APR timedelta Liq + 200',
                                                               'APR timedelta Liq + 500',
                                                               'APR timedelta Liq + 1000',                   
                                                               'APR timedelta Liq + 2000',
                                                               'APR timedelta Liq - 200',
                                                               'APR timedelta Liq - 500',
                                                               'APR timedelta Liq - 1000',                   
                                                               'APR timedelta Liq - 2000',                       
                                                                'APR yday',
                                                                'APR yday Liq + 200',
                                                               'APR yday Liq + 500',
                                                               'APR yday Liq + 1000',                   
                                                               'APR yday Liq + 2000',
                                                               'APR yday Liq - 200',
                                                               'APR yday Liq - 500',
                                                               'APR yday Liq - 1000',                   
                                                               'APR yday Liq - 2000']].round(2)

df_final_combined[['Liquidity timedelta',
                   'Liquidity yday',
                   'Volume yday']] = df_final_combined[['Liquidity timedelta',
                                                        'Liquidity yday',
                                                        'Volume yday']].fillna(0).astype(int)

df_final_combined['APR timedelta'] = df_final_combined['APR timedelta'].round(1)

criteria = (df_final_combined['Volume yday'] > 0) & (df_final_combined['APR timedelta'] > 0)

df_final_combined.loc[criteria, :].dropna(subset = ['date'])

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[DodoSwap MATIC] WMATIC-USDC,2021-07-13,1340.1,1331.72,1319.29,1299.07,1260.44,1348.67,1361.67,1383.90,1430.60,1260.87,1236.13,1200.78,1146.16,1050.57,1286.62,1327.29,1401.10,1576.43,31629,9991,287622
[Sushiswap MATIC] WMATIC-QUICK,2021-07-13,96.4,81.82,66.67,50.95,34.63,117.37,174.10,895.77,-122.88,99.06,84.03,68.45,52.29,35.52,120.65,179.23,939.66,-125.53,1120,1117,1011
[Sushiswap MATIC] WMATIC-USDT,2021-07-13,67.0,63.37,58.60,52.06,42.56,71.10,78.25,94.02,157.49,69.43,65.68,60.76,54.02,44.21,73.63,80.98,97.14,161.67,3481,3505,2222
[Sushiswap MATIC] WMATIC-USDC,2021-07-13,58.5,58.54,58.53,58.52,58.51,58.54,58.55,58.55,58.57,41.20,41.20,41.19,41.19,41.18,41.20,41.20,41.21,41.22,3989226,4069801,1531197
[Sushiswap MATIC] WMATIC-FRAX,2021-07-13,68.4,49.97,35.62,24.08,14.62,108.07,843.85,-81.56,-25.54,22.82,16.66,11.86,8.01,4.86,36.19,299.54,-26.92,-8.47,544,541,112
[ApeSwap MATIC] USDC-SNX,2021-07-13,46.3,46.13,45.92,45.58,44.90,46.41,46.63,46.99,47.73,7.40,7.38,7.35,7.30,7.19,7.43,7.46,7.52,7.63,65607,67566,6853
[Quickswap MATIC] WMATIC-USDT,2021-07-13,44.6,44.54,44.52,44.49,44.42,44.56,44.58,44.61,44.68,49.71,49.70,49.68,49.64,49.57,49.73,49.75,49.78,49.86,700417,687939,312313
[SpiritSwap FTM] WFTM-DAI,2021-07-13,56.6,44.33,33.45,23.74,15.02,78.29,183.99,-147.15,-31.99,62.11,48.58,36.61,25.96,16.41,86.08,204.52,-158.17,-34.79,722,718,407
[Sushiswap MATIC] WMATIC-DAI,2021-07-13,43.5,41.26,38.34,34.30,28.33,45.91,50.16,59.30,93.31,42.26,39.87,36.74,32.50,26.40,44.97,49.74,60.43,105.97,3743,3326,1284
